# Hybrid Execution Automatic Switching Demo

In [1]:
import snowflake.snowpark.modin.plugin
import modin.pandas as pd
import numpy as np
import datetime
import pandas as native_pd
from snowflake.snowpark.session import Session; session = Session.builder.create()

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
Going to open: https://snowbiz.okta.com/app/snowflake/exk8wfsfryJIn4IWZ2p7/sso/saml?SAMLRequest=jVJdc9owEPwrHvXZluyYCdVgMiaUiTspeDAkLW%2FCFqDBllydjKG%2FvjIfnfQhmbxpTru3e7c3eDhWpXPgGoSSEfI9ghwuc1UIuY3QcjFx%2B8gBw2TBSiV5hE4c0MNwAKwqaxo3Zifn%2FHfDwTi2kQTafUSo0ZIqBgKoZBUHanKaxT%2BeaeARygC4NlYOXSkFCKu1M6amGLdt67V3ntJbHBBCMPmKLaqDfEFvJOqPNWqtjMpVeaMc7UzvSPiYhJ2ERViF9EocCXlZwUcq6wsI6NNikbrpLFsgJ75N96gkNBXXGdcHkfPl%2FPliAKyDbDp7fZots28eSNVuSrbnuarqxthunn3hDS9wqbbC7igZR6jei2Kd6l7%2F56GYhqNdtpo36%2FvjNma%2FwjScczNK%2FRXbsckp2839fo6cl1uiQZdoAtDwRHY5GlsiQc8loev7C0IoCWgv9O5IsELO2OYoJDNn5s1sZ3Et%2Fnhqb9jZHKtr%2FM835sd9v93ARp%2B%2BJzJMXldBfY8BFO5iQpdLoWcDevjZ%2BQf4Let6bFO7%2F2ScqlLkJ2eidMXM%2B%2FH4nn%2BuiMLdnKGUV0yUcVFoDmBjKkvVPmrOjL1poxuO8PCi%2Bv9VD%2F8C&RelayState=ver%

## Example 1: Working with small/inline-created dataframe is faster

In [2]:
us_holidays = [
    ("New Year's Day", "2025-01-01"),
    ("Martin Luther King Jr. Day", "2025-01-20"),
    ("Presidents' Day", "2025-02-17"),
    ("Memorial Day", "2025-05-26"),
    ("Juneteenth National Independence Day", "2025-06-19"),
    ("Independence Day", "2025-07-04"),
    ("Labor Day", "2025-09-01"),
    ("Columbus Day", "2025-10-13"),
    ("Veterans Day", "2025-11-11"),
    ("Thanksgiving Day", "2025-11-27"),
    ("Christmas Day", "2025-12-25")
]

# Create DataFrame
df_us_holidays = pd.DataFrame(us_holidays, columns=["Holiday", "Date"])

# Convert Date column to datetime
df_us_holidays["Date"] = pd.to_datetime(df_us_holidays["Date"])

In [3]:
assert df_us_holidays.get_backend() == 'Pandas'  # with auto, we should expect this to be local

In [4]:
# Add new columns for transformations
df_us_holidays["Day_of_Week"] = df_us_holidays["Date"].dt.day_name()
df_us_holidays["Month"] = df_us_holidays["Date"].dt.month_name()

In [5]:
df_us_holidays

,Holiday,Date,Day_of_Week,Month
0,New Year's Day,2025-01-01,Wednesday,January
1,Martin Luther King Jr. Day,2025-01-20,Monday,January
2,Presidents' Day,2025-02-17,Monday,February
3,Memorial Day,2025-05-26,Monday,May
4,Juneteenth National Independence Day,2025-06-19,Thursday,June
5,Independence Day,2025-07-04,Friday,July
6,Labor Day,2025-09-01,Monday,September
7,Columbus Day,2025-10-13,Monday,October
8,Veterans Day,2025-11-11,Tuesday,November
9,Thanksgiving Day,2025-11-27,Thursday,November


In [6]:
%%time
#Note that without auto-switching, this took 2.5 min
for index, row in df_us_holidays.iterrows():
    print(f"{row['Holiday']} falls on {row['Day_of_Week']}, {row['Month']} {row['Date'].day}, {row['Date'].year}.")

New Year's Day falls on Wednesday, January 1, 2025.
Martin Luther King Jr. Day falls on Monday, January 20, 2025.
Presidents' Day falls on Monday, February 17, 2025.
Memorial Day falls on Monday, May 26, 2025.
Juneteenth National Independence Day falls on Thursday, June 19, 2025.
Independence Day falls on Friday, July 4, 2025.
Labor Day falls on Monday, September 1, 2025.
Columbus Day falls on Monday, October 13, 2025.
Veterans Day falls on Tuesday, November 11, 2025.
Thanksgiving Day falls on Thursday, November 27, 2025.
Christmas Day falls on Thursday, December 25, 2025.
CPU times: user 123 ms, sys: 5.59 ms, total: 128 ms
Wall time: 127 ms


Automatic engine switch happens when merged with large dataset.

In [7]:
df_transactions = pd.read_snowflake("REVENUE_TRANSACTIONS")

In [8]:
df_transactions["DATE"] = pd.to_datetime(df_transactions["DATE"])

In [9]:
len(df_us_holidays), len(df_transactions)

(11, 10000000)

In [10]:
combined = pd.merge(df_us_holidays, df_transactions, left_on="Date", right_on="DATE")

BackendCostCalculator Results: Pandas:1000/1000,*Snowflake:750/10000000


Transferring data from Pandas to Snowflake ...:   0%|          | 0/2 [00:00<?, ?it/s]

In [11]:
assert combined.get_backend() == 'Snowflake'

### 💡 Automatic switching speeds up loops/iterations on small data + inline creation of dataframes

## Example 2: When data is filtered the choice of engine changes

Run the following SQL to generate a synthetic dataset with 10M rows of transactions (from 2024-2025 current date)
```sql
CREATE OR REPLACE TABLE revenue_transactions (
    Transaction_ID STRING,
    Date DATE,
    Revenue FLOAT
);

SET num_days = (SELECT DATEDIFF(DAY, '2024-01-01', CURRENT_DATE));
INSERT INTO revenue_transactions (Transaction_ID, Date, Revenue)
SELECT
    UUID_STRING() AS Transaction_ID,
    DATEADD(DAY, UNIFORM(0, $num_days, RANDOM()), '2024-01-01') AS Date,
    UNIFORM(10, 1000, RANDOM()) AS Revenue
FROM TABLE(GENERATOR(ROWCOUNT => 10000000));
```

In [12]:
# Run the following to generate a synthetic dataset with 10M rows of transactions (from 2024-2025 current date)
session.sql('''
CREATE OR REPLACE TABLE revenue_transactions (
    Transaction_ID STRING,
    Date DATE,
    Revenue FLOAT
);''').collect()
session.sql('''SET num_days = (SELECT DATEDIFF(DAY, '2024-01-01', CURRENT_DATE));''').collect()
session.sql('''INSERT INTO revenue_transactions (Transaction_ID, Date, Revenue)
SELECT
    UUID_STRING() AS Transaction_ID,
    DATEADD(DAY, UNIFORM(0, $num_days, RANDOM()), '2024-01-01') AS Date,
    UNIFORM(10, 1000, RANDOM()) AS Revenue
FROM TABLE(GENERATOR(ROWCOUNT => 10000000));
''').collect()

[Row(number of rows inserted=10000000)]

In [13]:
df_transactions = pd.read_snowflake("REVENUE_TRANSACTIONS")

In [14]:
len(df_transactions)

10000000

Perform some operations on 10M rows with Snowflake

In [15]:
df_transactions["DATE"] = pd.to_datetime(df_transactions["DATE"])

In [16]:
df_transactions.groupby("DATE").sum()["REVENUE"]

DATE
2024-01-01    10899202.0
2024-01-02    10910864.0
2024-01-03    10882661.0
2024-01-04    10782880.0
2024-01-05    10864255.0
                 ...    
2025-04-07    10901511.0
2025-04-08    10843291.0
2025-04-09    10813787.0
2025-04-10    10860048.0
2025-04-11    10838760.0
Freq: None, Name: REVENUE, Length: 467, dtype: float64

In [17]:
assert df_transactions.get_backend() == "Snowflake"

So far everything has been happening in Snowflake, since we are working with the full dataset (10M rows). 
Next, we demonstrate what happens when we filter the data down to a smaller dataset below our 500k threshold for automatic switching. 
We showcase two was of doing the filtering: 
- Method 1: Filtering with pandas (lazy evaluation)
- Method 2: Prefilter with SQL during dataframe creation 

In [18]:
df_transactions_filter1 = df_transactions[(df_transactions["DATE"] >= pd.Timestamp.today().date() - pd.Timedelta('7 days')) & (df_transactions["DATE"] < pd.Timestamp.today().date())]

In [19]:
df_transactions_filter1.get_backend()

'Snowflake'

In [20]:
df_transactions_filter1._query_compiler._modin_frame.ordered_dataframe.row_count_upper_bound

1000000000000000000000000000000000000000000

In this case, since the data is already in Snowflake, it stays in Snowflake even after the filtering.

In [21]:
df_transactions_filter1

,TRANSACTION_ID,DATE,REVENUE
64,f0b0fb94-5297-45fd-bf2a-394838ecf415,2025-04-05,776.0
84,4959ae09-b348-4dfe-91ee-d9792ce6fafc,2025-04-06,910.0
275,b7d12028-d4d1-4754-a115-d9c37ee7d21b,2025-04-06,116.0
316,e13cb904-b16f-4157-bbc7-25c4fc9d624b,2025-04-09,211.0
389,3ed91e87-8486-43d4-a08b-0106e0359259,2025-04-08,501.0
...,...,...,...
9999819,efae70b2-2bc5-464e-8c98-cf70307587ac,2025-04-06,834.0
9999886,6d9ae1a0-7819-4acc-8ae5-100c669bb2ed,2025-04-05,377.0
9999898,8da525a8-5f6d-46db-ba20-d7601ef1c7cb,2025-04-05,699.0
9999975,6ac8949a-da0b-4755-9eec-d107ab17c8c7,2025-04-08,972.0


In [22]:
df_transactions_filter1._query_compiler._modin_frame.ordered_dataframe.row_count_upper_bound

150309

In [23]:
df_transactions_filter1

,TRANSACTION_ID,DATE,REVENUE
64,f0b0fb94-5297-45fd-bf2a-394838ecf415,2025-04-05,776.0
84,4959ae09-b348-4dfe-91ee-d9792ce6fafc,2025-04-06,910.0
275,b7d12028-d4d1-4754-a115-d9c37ee7d21b,2025-04-06,116.0
316,e13cb904-b16f-4157-bbc7-25c4fc9d624b,2025-04-09,211.0
389,3ed91e87-8486-43d4-a08b-0106e0359259,2025-04-08,501.0
...,...,...,...
9999819,efae70b2-2bc5-464e-8c98-cf70307587ac,2025-04-06,834.0
9999886,6d9ae1a0-7819-4acc-8ae5-100c669bb2ed,2025-04-05,377.0
9999898,8da525a8-5f6d-46db-ba20-d7601ef1c7cb,2025-04-05,699.0
9999975,6ac8949a-da0b-4755-9eec-d107ab17c8c7,2025-04-08,972.0


In [24]:
# Repr should 
# (1) perform the repr
# (2) update count on original data frame
# (3) consider moving
df_transactions_filter1 

,TRANSACTION_ID,DATE,REVENUE
64,f0b0fb94-5297-45fd-bf2a-394838ecf415,2025-04-05,776.0
84,4959ae09-b348-4dfe-91ee-d9792ce6fafc,2025-04-06,910.0
275,b7d12028-d4d1-4754-a115-d9c37ee7d21b,2025-04-06,116.0
316,e13cb904-b16f-4157-bbc7-25c4fc9d624b,2025-04-09,211.0
389,3ed91e87-8486-43d4-a08b-0106e0359259,2025-04-08,501.0
...,...,...,...
9999819,efae70b2-2bc5-464e-8c98-cf70307587ac,2025-04-06,834.0
9999886,6d9ae1a0-7819-4acc-8ae5-100c669bb2ed,2025-04-05,377.0
9999898,8da525a8-5f6d-46db-ba20-d7601ef1c7cb,2025-04-05,699.0
9999975,6ac8949a-da0b-4755-9eec-d107ab17c8c7,2025-04-08,972.0


In [25]:
# Do we want to check this eager
# Do we expect that after a filter we check for a switcheroo case?
# can we even do this move if the estimated row size is still the size of data table
assert df_transactions_filter1.get_backend() == "Snowflake" 

In [26]:
print(f"Date range: {df_transactions_filter1['DATE'].min().date()} to {df_transactions_filter1['DATE'].max().date()}. Resulting dataset size: {len(df_transactions_filter1)}")

Date range: 2025-04-03 to 2025-04-09. Resulting dataset size: 150309


Now let's perform filtering via SQL directly, so the dataframe upon creation is small.

In [27]:
df_transactions_filter2 = pd.read_snowflake("SELECT * FROM revenue_transactions WHERE Date >= DATEADD( 'days', -7, current_date ) and Date < current_date")

rcr 0.150309 type <class 'modin.core.storage_formats.pandas.native_query_compiler.NativeQueryCompiler'> cost: 150



Transferring data from Snowflake to Pandas ...:   0%|          | 0/2 [00:00<?, ?it/s]

In [28]:
df_transactions_filter2

,TRANSACTION_ID,DATE,REVENUE
0,e60175f9-d14c-4edc-a3d1-1be6dc0dece6,2025-04-03,960.0
1,179b94d5-acf3-4b69-95c0-266cfd1152a5,2025-04-04,444.0
2,311b828d-9b05-4035-8e7c-42d8b1c98b0e,2025-04-03,15.0
3,a5d39352-7518-4666-a5bc-39925447023c,2025-04-06,381.0
4,84137059-9918-4052-9438-1b41a555bc79,2025-04-07,498.0
...,...,...,...
150304,5f0172b9-f7ff-4742-aacf-121df89da7e4,2025-04-06,762.0
150305,797d7dc2-d549-486c-be08-39fe64c29d29,2025-04-03,981.0
150306,f8066adc-ca98-4d04-9787-be305e9d87c4,2025-04-08,401.0
150307,49d816e3-5f64-4c25-be31-e06ea6fd302e,2025-04-08,95.0


In [29]:
# Verify the result is same as above
print(f"Date range: {df_transactions_filter2['DATE'].min()} to {df_transactions_filter2['DATE'].max()}. Resulting dataset size: {len(df_transactions_filter2)}")

Date range: 2025-04-03 to 2025-04-09. Resulting dataset size: 150309


In [30]:
assert df_transactions_filter2.get_backend() == "Pandas" 

In [31]:
len(df_transactions_filter2)

150309

Once you are in pandas, you can still continue to perform the same operations: 

In [32]:
%time
df_transactions_filter1.groupby("DATE").sum()["REVENUE"]

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 7.87 µs
rcr 0.150309 type <class 'modin.core.storage_formats.pandas.native_query_compiler.NativeQueryCompiler'> cost: 150



Transferring data from Snowflake to Pandas ...:   0%|          | 0/2 [00:00<?, ?it/s]

DATE
2025-04-03    10804868.0
2025-04-04    10813828.0
2025-04-05    10911553.0
2025-04-06    10890288.0
2025-04-07    10720254.0
2025-04-08    10864018.0
2025-04-09    10922610.0
Freq: None, Name: REVENUE, dtype: float64

In [33]:
df_transactions_filter1.shape

(150309, 3)

In [34]:
%time
df_transactions_filter2.groupby("DATE").sum()["REVENUE"]

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 8.11 µs


DATE
2025-04-03    10804868.0
2025-04-04    10813828.0
2025-04-05    10911553.0
2025-04-06    10890288.0
2025-04-07    10720254.0
2025-04-08    10864018.0
2025-04-09    10922610.0
Name: REVENUE, dtype: float64

### 💡 Automatic switching means that pandas work well for both small and large data

## Example 3: Performing Custom `apply` on small dataset

Forecast using last year's transaction data via a custom apply function

In [35]:
start_date = pd.Timestamp("2025-10-01")
end_date = pd.Timestamp("2025-10-31")

In [36]:
# Forecasting function using df.apply

def forecast_revenue(df, start_date, end_date):
    # Filter data from last year
    df_filtered = df[(df["DATE"] >= start_date - pd.Timedelta(days=365)) & (df["DATE"] < start_date)]
    
    # Append future dates to daily_avg for prediction
    future_dates = pd.date_range(start=start_date, end=end_date, freq="D")
    df_future = pd.DataFrame({"DATE": future_dates})

    # Group by DATE and calculate the mean revenue
    daily_avg = df_filtered.groupby("DATE")["REVENUE"].mean().reset_index()
    daily_avg["DATE"] = daily_avg["DATE"].astype('datetime64[ns]')
    # Merge future dates with predicted revenue, filling missing values
    df_forecast = df_future.merge(daily_avg, on="DATE", how="left")
    #breakpoint()
    import numpy as np
    # Fill missing predicted revenue with overall mean from last year
    df_forecast["PREDICTED_REVENUE"] = np.nan
    df_forecast["PREDICTED_REVENUE"].fillna(daily_avg["REVENUE"].mean(), inplace=True)
    df_forecast["PREDICTED_REVENUE"] = df_forecast["PREDICTED_REVENUE"].astype("float")
    return df_forecast

In [37]:
df_forecast = forecast_revenue(df_transactions, start_date, end_date)
len(df_forecast)

rcr 0.000192 type <class 'modin.core.storage_formats.pandas.native_query_compiler.NativeQueryCompiler'> cost: 0



Transferring data from Snowflake to Pandas ...:   0%|          | 0/2 [00:00<?, ?it/s]

31

The resulting dataframe is very small, since it is only the 1-month window we're performing forecast on.

In [38]:
assert df_forecast.get_backend() == 'Pandas'

In [39]:
def adjust_for_holiday_weekend(row):
    # For national holidays, revenue down 5% since stores are closed. For weekends, revenue is up 5% due to increased activity.
    if row["DATE"].strftime('%Y-%m-%d') in list(df_us_holidays["Date"].dt.strftime('%Y-%m-%d')): 
        return row["PREDICTED_REVENUE"] * 0.95
    elif row["DATE"].weekday() == 5 or row["DATE"].weekday() == 6: #Saturday/Sundays
        return row["PREDICTED_REVENUE"] * 1.05
    return row["PREDICTED_REVENUE"]

In [40]:
# Adjust for holidays using the apply function
df_forecast["PREDICTED_REVENUE"] = df_forecast.apply(adjust_for_holiday_weekend, axis=1)
df_forecast[["DATE","PREDICTED_REVENUE"]]

,DATE,PREDICTED_REVENUE
0,2025-10-01,505.088929
1,2025-10-02,505.088929
2,2025-10-03,505.088929
3,2025-10-04,530.343376
4,2025-10-05,530.343376
5,2025-10-06,505.088929
6,2025-10-07,505.088929
7,2025-10-08,505.088929
8,2025-10-09,505.088929
9,2025-10-10,505.088929


In [41]:
assert df_forecast.get_backend() == 'Pandas'

In [42]:
print(f"Altair takes in {type(df_forecast)} with {df_forecast.get_backend()} as backend, since we implement the dataframe interchange protocol")

Altair takes in <class 'modin.pandas.dataframe.DataFrame'> with Pandas as backend, since we implement the dataframe interchange protocol


In [43]:
import altair as alt
alt.data_transformers.disable_max_rows()

chart_predicted = alt.Chart(df_forecast).mark_line(color='blue').encode(
    x='monthdate(DATE):T',
    y=alt.Y('PREDICTED_REVENUE:Q',scale=alt.Scale(domain=[470, 550])),
    tooltip=['DATE', 'PREDICTED_REVENUE']
)
chart_predicted

ValueError: invalid literal for int() with base 10: ''

alt.Chart(...)

In [44]:
df_transactions_filtered = df_transactions[
    (df_transactions["DATE"] >= start_date - pd.Timedelta(days=365)) &
    (df_transactions["DATE"] < end_date - pd.Timedelta(days=365))
]
df_transactions_filtered_groupby = df_transactions_filtered.groupby("DATE")["REVENUE"].mean().reset_index()

rcr 3e-05 type <class 'modin.core.storage_formats.pandas.native_query_compiler.NativeQueryCompiler'> cost: 0



Transferring data from Snowflake to Pandas ...:   0%|          | 0/2 [00:00<?, ?it/s]

In [45]:
print(f"Altair takes in {type(df_transactions_filtered_groupby)} with {df_transactions_filtered_groupby.get_backend()} as backend, since we implement the dataframe interchange protocol")

Altair takes in <class 'modin.pandas.dataframe.DataFrame'> with Pandas as backend, since we implement the dataframe interchange protocol


In [46]:
df_forecast_labeled = df_forecast.copy()
df_forecast_labeled['Label'] = 'Predicted Revenue'
df_forecast_labeled = df_forecast_labeled.rename(columns={'PREDICTED_REVENUE': 'Value'})

df_last_year_labeled = df_transactions_filtered_groupby.copy()
df_last_year_labeled['Label'] = 'Revenue'
df_last_year_labeled = df_last_year_labeled.rename(columns={'REVENUE': 'Value'})

# Combine
combined_df = pd.concat([
    df_forecast_labeled[['DATE', 'Value', 'Label']],
    df_last_year_labeled[['DATE', 'Value', 'Label']]
])

# Plot with Value on X and color based on Label
final_chart = alt.Chart(combined_df).mark_line().encode(
    y=alt.Y('Value:Q',scale=alt.Scale(domain=[470, 550])),
    x='monthdate(DATE):T',
    color=alt.Color('Label:N', legend=alt.Legend(title='Type')),
    tooltip=['DATE', 'Value', 'Label']
).properties(
    title='Revenue vs Predicted Revenue (by Value)'
)

final_chart

ValueError: invalid literal for int() with base 10: ''

alt.Chart(...)

### 💡 Apply on small dataset is much faster with automatic switching running with pandas locally.